In [1]:
!pip install transformers datasets accelerate scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.5 MB/s eta 0:00:00:00:0100:01
  Attempting u

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import json
import pandas as pd

2025-11-13 04:15:58.203313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763007358.379464      87 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763007358.424298      87 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# -------- LOAD DATA --------
with open("/kaggle/input/traindata/train_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)
dataset = dataset.rename_column("syllogism", "text")
dataset = dataset.rename_column("validity", "label")
dataset = dataset.remove_columns(["id", "plausibility"])
from datasets import Value
dataset = dataset.cast_column("label", Value("int64"))

#Clean text and ensure balanced labels
import re
from sklearn.utils import resample

def clean_text(text):
    text = re.sub(r'\s+', ' ', text.strip())   # remove extra spaces/newlines
    text = re.sub(r'[^A-Za-z0-9\s.,;!?]', '', text)  # basic cleanup
    return text

dataset = dataset.map(lambda x: {"text": clean_text(x["text"])})

# Balance labels if skewed
df = pd.DataFrame(dataset)
label_counts = df["label"].value_counts(normalize=True)
if abs(label_counts.get(0, 0) - label_counts.get(1, 0)) > 0.15:  # if imbalance >15%
    minority = df[df["label"] == df["label"].value_counts().idxmin()]
    majority = df[df["label"] == df["label"].value_counts().idxmax()]
    minority_upsampled = resample(minority, replace=True, n_samples=len(majority), random_state=42)
    df_balanced = pd.concat([majority, minority_upsampled])
    dataset = Dataset.from_pandas(df_balanced)

Casting the dataset:   0%|          | 0/960 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

In [4]:
# -------- TOKENIZATION --------
model_name = "microsoft/mdeberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

encoded = dataset.map(tokenize_fn, batched=True)
encoded = encoded.train_test_split(test_size=0.1, seed=42)

train_ds = encoded["train"]
val_ds = encoded["test"]

cols = ["input_ids", "attention_mask", "label"]
train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(type="torch", columns=cols)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/960 [00:00<?, ? examples/s]

In [5]:
# -------- MODEL --------
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# -------- TRAINING ARGS (simple, safe) --------
args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=1e-5,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=50,       # print every 50 steps
    report_to="none",       # disables wandb noise
)


pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Metric functions

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'eval_accuracy': acc,
        'eval_precision': precision,
        'eval_recall': recall,
        'eval_f1': f1
    }

In [7]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA available: True
Device name: Tesla P100-PCIE-16GB


In [8]:
model.to("cuda")  # force model to GPU

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [9]:
# -------- TRAINER --------
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_87/3581206564.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
# -------- TRAIN + EVALUATE --------
from transformers import TrainerCallback

# Custom callback to print live progress
class PrintProgressCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            print(f"Step {state.global_step} | Epoch {state.epoch:.2f} | Loss: {logs['loss']:.4f}")

# Attach callback
trainer.add_callback(PrintProgressCallback)

print("Starting training...\n")
train_output = trainer.train()   # run once

# Epoch-wise training loss summary
print("\nEpoch-wise Training Summary:")
epoch_losses = {}
for log in trainer.state.log_history:
    if "loss" in log and "epoch" in log:
        epoch = int(round(log["epoch"]))
        epoch_losses.setdefault(epoch, []).append(log["loss"])

for e, losses in epoch_losses.items():
    print(f"  Epoch {e} | Avg Loss: {sum(losses)/len(losses):.4f}")


Starting training...



Step,Training Loss
50,0.694300
100,0.691200
150,0.615300
200,0.546000
250,0.436900
300,0.463000
350,0.359100
400,0.321300
450,0.331200
500,0.242500


Step 50 | Epoch 0.46 | Loss: 0.6943
Step 100 | Epoch 0.93 | Loss: 0.6912
Step 150 | Epoch 1.39 | Loss: 0.6153
Step 200 | Epoch 1.85 | Loss: 0.5460
Step 250 | Epoch 2.31 | Loss: 0.4369
Step 300 | Epoch 2.78 | Loss: 0.4630
Step 350 | Epoch 3.24 | Loss: 0.3591
Step 400 | Epoch 3.70 | Loss: 0.3213
Step 450 | Epoch 4.17 | Loss: 0.3312
Step 500 | Epoch 4.63 | Loss: 0.2425

Epoch-wise Training Summary:
  Epoch 0 | Avg Loss: 0.6943
  Epoch 1 | Avg Loss: 0.6532
  Epoch 2 | Avg Loss: 0.4915
  Epoch 3 | Avg Loss: 0.4111
  Epoch 4 | Avg Loss: 0.3262
  Epoch 5 | Avg Loss: 0.2425


In [11]:
# Evaluate after training
print("\nEvaluating on training data...")
train_metrics = trainer.evaluate(train_ds)
print(train_metrics)

print("\nEvaluating on validation data...")
val_metrics = trainer.evaluate(val_ds)
print(val_metrics)

print("\nFinal Validation Summary:")
print(f"  Accuracy: {val_metrics['eval_accuracy']:.4f}")
print(f"  F1 Score: {val_metrics['eval_f1']:.4f}")
print(f"  Precision: {val_metrics['eval_precision']:.4f}")
print(f"  Recall: {val_metrics['eval_recall']:.4f}")



Evaluating on training data...


{'eval_accuracy': 0.9317129629629629, 'eval_precision': 0.9481481481481482, 'eval_recall': 0.909952606635071, 'eval_f1': 0.9286577992744861, 'eval_loss': 0.1980215460062027, 'eval_runtime': 6.276, 'eval_samples_per_second': 137.668, 'eval_steps_per_second': 34.417, 'epoch': 5.0}

Evaluating on validation data...
{'eval_accuracy': 0.8333333333333334, 'eval_precision': 0.9038461538461539, 'eval_recall': 0.8103448275862069, 'eval_f1': 0.8545454545454546, 'eval_loss': 0.47993218898773193, 'eval_runtime': 0.701, 'eval_samples_per_second': 136.949, 'eval_steps_per_second': 34.237, 'epoch': 5.0}

Final Validation Summary:
  Accuracy: 0.8333
  F1 Score: 0.8545
  Precision: 0.9038
  Recall: 0.8103
